In [5]:
import numpy as np
import pandas as pd
import warnings
import datetime
warnings.simplefilter('ignore')
import math
import time
from statistics import mean
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from sklearn.metrics import mean_squared_error


#from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import category_encoders as ce

In [2]:
status = pd.read_csv('../../../status.csv')
#statusのyear, month, dayを結合してdatetime型に
status['date'] = status['year'].astype(str) + '/' + status['month'].astype(str).str.zfill(2).astype(str) + '/' + status['day'].astype(str).str.zfill(2).astype(str) + '/' + status['hour'].astype(str).str.zfill(2).astype(str)
status['date'] = pd.to_datetime(status['date'], format='%Y/%m/%d/%H', infer_datetime_format=True)

#曜日を追加するための関数を定義
def get_weekday_jp(dt):
    w_list = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']
    return(w_list[dt.weekday()])

#dateから曜日情報を取得
status["weekday"] = status["date"].apply(get_weekday_jp)

main_df = status[['date','hour', 'station_id', 'bikes_available', 'weekday', 'day']]

#カテゴリ変数をダミー変数化
main_df = pd.get_dummies(main_df)

stationid_set = [0, 15, 32, 36, 38, 40, 43, 49, 54, 56] 
dt_now = datetime.datetime.now()
print(dt_now)

2022-01-20 22:58:11.620594


In [3]:
for station_id in stationid_set:
    train = main_df[main_df['station_id'] == station_id]
    train = train[train['date'] < '2013-10-01']
    train = train[train['date'] >= '2013-09-01']
    train = train.drop("station_id",axis=1)

    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-09-30']
    test = test.drop("station_id",axis=1)

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    #0から1の範囲に値を正規化して配列に直しているのが下の2行
    train_df_scale = scaler.fit_transform(train.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(train[["bikes_available"]])
    
    length = len(train_df_scale)
    train = train_df_scale[0:length,:]

    def create_dataset(dataset):
        dataX = []
        dataY = np.array([])

        #trainまたはtestの行を計算している。
        max_len = len(dataset)
        for i in range(24,max_len):
            xset = []
            #trainまたはtestの列を繰り返しに入れている。
            for j in range(1, dataset.shape[1]):
                a = dataset[i-24:i, j]
                xset.append(a)
            #各行の正規化したバイク台数を取ってくる
            temp_array = np.array(dataset[i:i+1,0])
            #各行からバイク台数だけを取ってきたものをまとめている
            dataY = np.concatenate([dataY,temp_array])
            dataX.append(xset)
            #dataYの形を変えてるだけ
        dataY = dataY.reshape(-1,1)
        return np.array(dataX), dataY 

    trainX, trainY = create_dataset(train)
    #LSTMのモデルに入力用にデータの形を整形
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))

    model = Sequential()
    model.add(LSTM(50, input_shape=(trainX.shape[1],24)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    hist = model.fit(trainX, trainY, epochs=300, batch_size=5, verbose=2)
    
    #学習済みモデルで予測
    train_predict = model.predict(trainX)
    #スケールをもとに戻す
    train_predict = scaler_for_inverse.inverse_transform(train_predict)
    trainY = scaler_for_inverse.inverse_transform(trainY)
    #各ステーションのスコアの平均値を算出
    train_score_list = []
    trainscore = math.sqrt(mean_squared_error(trainY[:,0], train_predict[:,0]))
    train_score_list.append(trainscore)   
    
    array = []
    for i in train_predict:
        array.append(i[0])

    train = main_df[main_df['station_id'] == 0]
    train = train[train['date'] < '2013-10-01']
    train = train[train['date'] >= '2013-09-02']
    
    train["predict"] =  array
    train_file_name = "LSTM_24_predict/lstm_train_stationid" + str(station_id) + "_201309.csv" 
    train.to_csv(train_file_name) 

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    test_df_scale = scaler.fit_transform(test.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(test[["bikes_available"]])
    
    length = len(test_df_scale)
    test = test_df_scale[0:length,:]
    testX, testY = create_dataset(test)
    #LSTMのモデルに入力用にデータの形を整形
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
    #学習済みモデルで予測
    test_predict = model.predict(testX)
    #スケールをもとに戻す
    test_predict = scaler_for_inverse.inverse_transform(test_predict)
    testY = scaler_for_inverse.inverse_transform(testY)
    #各ステーションのスコアの平均値を算出
    test_score_list = []
    testscore = math.sqrt(mean_squared_error(testY[:,0], test_predict[:,0]))
    test_score_list.append(testscore)

    array_pred = []
    for i in test_predict:
        array_pred.append(i[0])    
        
    test = main_df[main_df['station_id'] == 0]
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-10-01']

    test['predict'] = array_pred
    test_file_name = "LSTM_24_predict/lstm_test_stationid" + str(station_id) + "_201309.csv" 
    test.to_csv(test_file_name)
    
    dt_now = datetime.datetime.now()
    print(dt_now)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                15000     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 15,051
Trainable params: 15,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
140/140 - 1s - loss: 0.0434
Epoch 2/300
140/140 - 0s - loss: 0.0260
Epoch 3/300
140/140 - 0s - loss: 0.0246
Epoch 4/300
140/140 - 0s - loss: 0.0232
Epoch 5/300
140/140 - 0s - loss: 0.0227
Epoch 6/300
140/140 - 0s - loss: 0.0219
Epoch 7/300
140/140 - 0s - loss: 0.0205
Epoch 8/300
140/140 - 0s - loss: 0.0193
Epoch 9/300
140/140 - 0s - loss: 0.0184
Epoch 10/300
140/140 - 0s - loss: 0.0174
Epoch 11/300
140/140 - 0s - loss: 0.0173
Epoch 12/300
140/140 - 0s - loss: 0.0166
Epoch 13/300
140/140 - 0s - los

Epoch 178/300
140/140 - 0s - loss: 3.7359e-04
Epoch 179/300
140/140 - 0s - loss: 3.9722e-04
Epoch 180/300
140/140 - 0s - loss: 3.4160e-04
Epoch 181/300
140/140 - 0s - loss: 4.0889e-04
Epoch 182/300
140/140 - 0s - loss: 3.1773e-04
Epoch 183/300
140/140 - 0s - loss: 3.4004e-04
Epoch 184/300
140/140 - 0s - loss: 4.0344e-04
Epoch 185/300
140/140 - 0s - loss: 4.1167e-04
Epoch 186/300
140/140 - 0s - loss: 4.4629e-04
Epoch 187/300
140/140 - 0s - loss: 3.2489e-04
Epoch 188/300
140/140 - 0s - loss: 4.1654e-04
Epoch 189/300
140/140 - 0s - loss: 5.0822e-04
Epoch 190/300
140/140 - 0s - loss: 3.8799e-04
Epoch 191/300
140/140 - 0s - loss: 2.8119e-04
Epoch 192/300
140/140 - 0s - loss: 2.6899e-04
Epoch 193/300
140/140 - 0s - loss: 2.5427e-04
Epoch 194/300
140/140 - 0s - loss: 2.6589e-04
Epoch 195/300
140/140 - 0s - loss: 4.2153e-04
Epoch 196/300
140/140 - 0s - loss: 3.2228e-04
Epoch 197/300
140/140 - 0s - loss: 2.5929e-04
Epoch 198/300
140/140 - 0s - loss: 3.2301e-04
Epoch 199/300
140/140 - 0s - loss:

Epoch 48/300
140/140 - 0s - loss: 0.0038
Epoch 49/300
140/140 - 0s - loss: 0.0036
Epoch 50/300
140/140 - 0s - loss: 0.0036
Epoch 51/300
140/140 - 0s - loss: 0.0118
Epoch 52/300
140/140 - 0s - loss: 0.0031
Epoch 53/300
140/140 - 0s - loss: 0.0029
Epoch 54/300
140/140 - 0s - loss: 0.0026
Epoch 55/300
140/140 - 0s - loss: 0.0028
Epoch 56/300
140/140 - 0s - loss: 0.0024
Epoch 57/300
140/140 - 0s - loss: 0.0028
Epoch 58/300
140/140 - 0s - loss: 0.0026
Epoch 59/300
140/140 - 0s - loss: 0.0022
Epoch 60/300
140/140 - 0s - loss: 0.0018
Epoch 61/300
140/140 - 0s - loss: 0.0022
Epoch 62/300
140/140 - 0s - loss: 0.0017
Epoch 63/300
140/140 - 0s - loss: 0.0019
Epoch 64/300
140/140 - 0s - loss: 0.0042
Epoch 65/300
140/140 - 0s - loss: 0.0019
Epoch 66/300
140/140 - 0s - loss: 0.0016
Epoch 67/300
140/140 - 0s - loss: 0.0017
Epoch 68/300
140/140 - 0s - loss: 0.0019
Epoch 69/300
140/140 - 0s - loss: 0.0016
Epoch 70/300
140/140 - 0s - loss: 0.0018
Epoch 71/300
140/140 - 0s - loss: 0.0021
Epoch 72/300
140

Epoch 233/300
140/140 - 0s - loss: 1.4005e-04
Epoch 234/300
140/140 - 0s - loss: 2.1327e-04
Epoch 235/300
140/140 - 0s - loss: 1.7227e-04
Epoch 236/300
140/140 - 0s - loss: 1.4680e-04
Epoch 237/300
140/140 - 0s - loss: 1.1894e-04
Epoch 238/300
140/140 - 0s - loss: 1.7440e-04
Epoch 239/300
140/140 - 0s - loss: 1.6672e-04
Epoch 240/300
140/140 - 0s - loss: 2.0509e-04
Epoch 241/300
140/140 - 0s - loss: 2.0071e-04
Epoch 242/300
140/140 - 0s - loss: 1.7211e-04
Epoch 243/300
140/140 - 0s - loss: 1.8053e-04
Epoch 244/300
140/140 - 0s - loss: 1.3111e-04
Epoch 245/300
140/140 - 0s - loss: 0.0010
Epoch 246/300
140/140 - 0s - loss: 0.0041
Epoch 247/300
140/140 - 0s - loss: 8.5193e-04
Epoch 248/300
140/140 - 0s - loss: 4.5684e-04
Epoch 249/300
140/140 - 0s - loss: 2.1306e-04
Epoch 250/300
140/140 - 0s - loss: 7.2746e-04
Epoch 251/300
140/140 - 0s - loss: 7.9475e-04
Epoch 252/300
140/140 - 0s - loss: 4.8054e-04
Epoch 253/300
140/140 - 0s - loss: 1.2428e-04
Epoch 254/300
140/140 - 0s - loss: 9.5454e

Epoch 109/300
140/140 - 0s - loss: 0.0018
Epoch 110/300
140/140 - 0s - loss: 0.0021
Epoch 111/300
140/140 - 0s - loss: 0.0020
Epoch 112/300
140/140 - 0s - loss: 0.0017
Epoch 113/300
140/140 - 0s - loss: 0.0016
Epoch 114/300
140/140 - 0s - loss: 0.0016
Epoch 115/300
140/140 - 0s - loss: 0.0018
Epoch 116/300
140/140 - 0s - loss: 0.0019
Epoch 117/300
140/140 - 0s - loss: 0.0018
Epoch 118/300
140/140 - 0s - loss: 0.0015
Epoch 119/300
140/140 - 0s - loss: 0.0016
Epoch 120/300
140/140 - 0s - loss: 0.0016
Epoch 121/300
140/140 - 0s - loss: 0.0015
Epoch 122/300
140/140 - 0s - loss: 0.0023
Epoch 123/300
140/140 - 0s - loss: 0.0018
Epoch 124/300
140/140 - 0s - loss: 0.0015
Epoch 125/300
140/140 - 0s - loss: 0.0014
Epoch 126/300
140/140 - 0s - loss: 0.0014
Epoch 127/300
140/140 - 0s - loss: 0.0014
Epoch 128/300
140/140 - 0s - loss: 0.0015
Epoch 129/300
140/140 - 0s - loss: 0.0014
Epoch 130/300
140/140 - 0s - loss: 0.0013
Epoch 131/300
140/140 - 0s - loss: 0.0015
Epoch 132/300
140/140 - 0s - loss:

...

Epoch 36/300
140/140 - 0s - loss: 0.0136
Epoch 37/300
140/140 - 0s - loss: 0.0124
Epoch 38/300
140/140 - 0s - loss: 0.0114
Epoch 39/300
140/140 - 0s - loss: 0.0109
Epoch 40/300
140/140 - 0s - loss: 0.0105
Epoch 41/300
140/140 - 0s - loss: 0.0103
Epoch 42/300
140/140 - 0s - loss: 0.0100
Epoch 43/300
140/140 - 0s - loss: 0.0094
Epoch 44/300
140/140 - 0s - loss: 0.0090
Epoch 45/300
140/140 - 0s - loss: 0.0091
Epoch 46/300
140/140 - 0s - loss: 0.0091
Epoch 47/300
140/140 - 0s - loss: 0.0086
Epoch 48/300
140/140 - 0s - loss: 0.0081
Epoch 49/300
140/140 - 0s - loss: 0.0075
Epoch 50/300
140/140 - 0s - loss: 0.0089
Epoch 51/300
140/140 - 0s - loss: 0.0087
Epoch 52/300
140/140 - 0s - loss: 0.0078
Epoch 53/300
140/140 - 0s - loss: 0.0070
Epoch 54/300
140/140 - 0s - loss: 0.0071
Epoch 55/300
140/140 - 0s - loss: 0.0081
Epoch 56/300
140/140 - 0s - loss: 0.0069
Epoch 57/300
140/140 - 0s - loss: 0.0067
Epoch 58/300
140/140 - 0s - loss: 0.0060
Epoch 59/300
140/140 - 0s - loss: 0.0062
Epoch 60/300
140

Epoch 227/300
140/140 - 0s - loss: 5.3434e-04
Epoch 228/300
140/140 - 0s - loss: 5.9250e-04
Epoch 229/300
140/140 - 0s - loss: 5.1766e-04
Epoch 230/300
140/140 - 0s - loss: 5.6142e-04
Epoch 231/300
140/140 - 0s - loss: 3.8285e-04
Epoch 232/300
140/140 - 0s - loss: 3.1428e-04
Epoch 233/300
140/140 - 0s - loss: 3.8161e-04
Epoch 234/300
140/140 - 0s - loss: 3.2576e-04
Epoch 235/300
140/140 - 0s - loss: 3.7219e-04
Epoch 236/300
140/140 - 0s - loss: 6.6855e-04
Epoch 237/300
140/140 - 0s - loss: 3.5523e-04
Epoch 238/300
140/140 - 0s - loss: 5.2186e-04
Epoch 239/300
140/140 - 0s - loss: 5.5409e-04
Epoch 240/300
140/140 - 0s - loss: 5.9252e-04
Epoch 241/300
140/140 - 0s - loss: 4.7838e-04
Epoch 242/300
140/140 - 0s - loss: 5.8035e-04
Epoch 243/300
140/140 - 0s - loss: 8.2758e-04
Epoch 244/300
140/140 - 0s - loss: 4.0570e-04
Epoch 245/300
140/140 - 0s - loss: 2.8418e-04
Epoch 246/300
140/140 - 0s - loss: 3.3724e-04
Epoch 247/300
140/140 - 0s - loss: 3.1969e-04
Epoch 248/300
140/140 - 0s - loss:

140/140 - 0s - loss: 0.0019
Epoch 103/300
140/140 - 0s - loss: 0.0017
Epoch 104/300
140/140 - 0s - loss: 0.0019
Epoch 105/300
140/140 - 0s - loss: 0.0017
Epoch 106/300
140/140 - 0s - loss: 0.0018
Epoch 107/300
140/140 - 0s - loss: 0.0020
Epoch 108/300
140/140 - 0s - loss: 0.0017
Epoch 109/300
140/140 - 0s - loss: 0.0018
Epoch 110/300
140/140 - 0s - loss: 0.0018
Epoch 111/300
140/140 - 0s - loss: 0.0019
Epoch 112/300
140/140 - 0s - loss: 0.0018
Epoch 113/300
140/140 - 0s - loss: 0.0016
Epoch 114/300
140/140 - 0s - loss: 0.0015
Epoch 115/300
140/140 - 0s - loss: 0.0016
Epoch 116/300
140/140 - 0s - loss: 0.0014
Epoch 117/300
140/140 - 0s - loss: 0.0015
Epoch 118/300
140/140 - 0s - loss: 0.0018
Epoch 119/300
140/140 - 0s - loss: 0.0015
Epoch 120/300
140/140 - 0s - loss: 0.0018
Epoch 121/300
140/140 - 0s - loss: 0.0014
Epoch 122/300
140/140 - 0s - loss: 0.0013
Epoch 123/300
140/140 - 0s - loss: 0.0013
Epoch 124/300
140/140 - 0s - loss: 0.0013
Epoch 125/300
140/140 - 0s - loss: 0.0013
Epoch 

140/140 - 0s - loss: 1.8668e-04
Epoch 285/300
140/140 - 0s - loss: 2.3593e-04
Epoch 286/300
140/140 - 0s - loss: 2.3627e-04
Epoch 287/300
140/140 - 0s - loss: 2.3253e-04
Epoch 288/300
140/140 - 0s - loss: 2.6093e-04
Epoch 289/300
140/140 - 0s - loss: 2.2073e-04
Epoch 290/300
140/140 - 0s - loss: 2.1802e-04
Epoch 291/300
140/140 - 0s - loss: 3.3810e-04
Epoch 292/300
140/140 - 0s - loss: 3.9175e-04
Epoch 293/300
140/140 - 0s - loss: 3.4449e-04
Epoch 294/300
140/140 - 0s - loss: 1.5306e-04
Epoch 295/300
140/140 - 0s - loss: 1.3052e-04
Epoch 296/300
140/140 - 0s - loss: 1.5106e-04
Epoch 297/300
140/140 - 0s - loss: 3.3627e-04
Epoch 298/300
140/140 - 0s - loss: 2.1786e-04
Epoch 299/300
140/140 - 0s - loss: 1.6217e-04
Epoch 300/300
140/140 - 0s - loss: 1.3262e-04
2022-01-12 06:44:36.874461
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 50)            

Epoch 161/300
140/140 - 0s - loss: 4.9387e-04
Epoch 162/300
140/140 - 0s - loss: 4.5704e-04
Epoch 163/300
140/140 - 0s - loss: 5.0128e-04
Epoch 164/300
140/140 - 0s - loss: 5.5915e-04
Epoch 165/300
140/140 - 0s - loss: 5.3344e-04
Epoch 166/300
140/140 - 0s - loss: 4.5063e-04
Epoch 167/300
140/140 - 0s - loss: 4.5169e-04
Epoch 168/300
140/140 - 0s - loss: 4.1537e-04
Epoch 169/300
140/140 - 0s - loss: 4.2880e-04
Epoch 170/300
140/140 - 0s - loss: 6.8482e-04
Epoch 171/300
140/140 - 0s - loss: 6.2444e-04
Epoch 172/300
140/140 - 0s - loss: 5.0344e-04
Epoch 173/300
140/140 - 0s - loss: 5.8597e-04
Epoch 174/300
140/140 - 0s - loss: 4.1849e-04
Epoch 175/300
140/140 - 0s - loss: 5.1947e-04
Epoch 176/300
140/140 - 0s - loss: 4.2911e-04
Epoch 177/300
140/140 - 0s - loss: 3.4846e-04
Epoch 178/300
140/140 - 0s - loss: 4.7646e-04
Epoch 179/300
140/140 - 0s - loss: 4.5693e-04
Epoch 180/300
140/140 - 0s - loss: 5.3573e-04
Epoch 181/300
140/140 - 0s - loss: 4.0991e-04
Epoch 182/300
140/140 - 0s - loss:

In [4]:
for station_id in stationid_set:
    train = main_df[main_df['station_id'] == station_id]
    train = train[train['date'] < '2014-02-01']
    train = train[train['date'] >= '2014-01-01']
    train = train.drop("station_id",axis=1)

    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2014-02-08 00:00:00']
    test = test[test['date'] >= '2014-01-31']
    test = test.drop("station_id",axis=1)

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    #0から1の範囲に値を正規化して配列に直しているのが下の2行
    train_df_scale = scaler.fit_transform(train.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(train[["bikes_available"]])
    
    length = len(train_df_scale)
    train = train_df_scale[0:length,:]

    def create_dataset(dataset):
        dataX = []
        dataY = np.array([])

        #trainまたはtestの行を計算している。
        max_len = len(dataset)
        for i in range(24,max_len):
            xset = []
            #trainまたはtestの列を繰り返しに入れている。
            for j in range(1, dataset.shape[1]):
                a = dataset[i-24:i, j]
                xset.append(a)
            #各行の正規化したバイク台数を取ってくる
            temp_array = np.array(dataset[i:i+1,0])
            #各行からバイク台数だけを取ってきたものをまとめている
            dataY = np.concatenate([dataY,temp_array])
            dataX.append(xset)
            #dataYの形を変えてるだけ
        dataY = dataY.reshape(-1,1)
        return np.array(dataX), dataY 

    trainX, trainY = create_dataset(train)
    #LSTMのモデルに入力用にデータの形を整形
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))

    model = Sequential()
    model.add(LSTM(50, input_shape=(trainX.shape[1],24)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    hist = model.fit(trainX, trainY, epochs=300, batch_size=5, verbose=2)
    
    #学習済みモデルで予測
    train_predict = model.predict(trainX)
    #スケールをもとに戻す
    train_predict = scaler_for_inverse.inverse_transform(train_predict)
    trainY = scaler_for_inverse.inverse_transform(trainY)
    #各ステーションのスコアの平均値を算出
    train_score_list = []
    trainscore = math.sqrt(mean_squared_error(trainY[:,0], train_predict[:,0]))
    train_score_list.append(trainscore)   
    
    array = []
    for i in train_predict:
        array.append(i[0])

    train = main_df[main_df['station_id'] == station_id]
    train = train[train['date'] < '2014-02-01']
    train = train[train['date'] >= '2014-01-02']
    
    train["predict"] =  array
    train_file_name = "LSTM_24_predict/lstm_train_stationid" + str(station_id) + "_201401.csv" 
    train.to_csv(train_file_name) 

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    test_df_scale = scaler.fit_transform(test.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(test[["bikes_available"]])
    
    length = len(test_df_scale)
    test = test_df_scale[0:length,:]
    testX, testY = create_dataset(test)
    #LSTMのモデルに入力用にデータの形を整形
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
    #学習済みモデルで予測
    test_predict = model.predict(testX)
    #スケールをもとに戻す
    test_predict = scaler_for_inverse.inverse_transform(test_predict)
    testY = scaler_for_inverse.inverse_transform(testY)
    #各ステーションのスコアの平均値を算出
    test_score_list = []
    testscore = math.sqrt(mean_squared_error(testY[:,0], test_predict[:,0]))
    test_score_list.append(testscore)

    array_pred = []
    for i in test_predict:
        array_pred.append(i[0])    
        
    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2014-02-08 00:00:00']
    test = test[test['date'] >= '2014-02-01']

    test['predict'] = array_pred
    test_file_name = "LSTM_24_predict/lstm_test_stationid" + str(station_id) + "_201401.csv" 
    test.to_csv(test_file_name) 

    dt_now = datetime.datetime.now()
    print(dt_now)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 50)                15000     
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 51        
Total params: 15,051
Trainable params: 15,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
144/144 - 1s - loss: 0.0556
Epoch 2/300
144/144 - 0s - loss: 0.0337
Epoch 3/300
144/144 - 0s - loss: 0.0237
Epoch 4/300
144/144 - 0s - loss: 0.0205
Epoch 5/300
144/144 - 0s - loss: 0.0198
Epoch 6/300
144/144 - 0s - loss: 0.0181
Epoch 7/300
144/144 - 0s - loss: 0.0172
Epoch 8/300
144/144 - 0s - loss: 0.0163
Epoch 9/300
144/144 - 0s - loss: 0.0171
Epoch 10/300
144/144 - 0s - loss: 0.0161
Epoch 11/300
144/144 - 0s - loss: 0.0149
Epoch 12/300
144/144 - 0s - loss: 0.0146
Epoch 13/300
144/144 - 0s - 

Epoch 180/300
144/144 - 0s - loss: 4.5067e-04
Epoch 181/300
144/144 - 0s - loss: 4.4499e-04
Epoch 182/300
144/144 - 0s - loss: 4.4092e-04
Epoch 183/300
144/144 - 0s - loss: 5.6246e-04
Epoch 184/300
144/144 - 0s - loss: 5.0762e-04
Epoch 185/300
144/144 - 0s - loss: 3.9574e-04
Epoch 186/300
144/144 - 0s - loss: 4.7283e-04
Epoch 187/300
144/144 - 0s - loss: 4.4686e-04
Epoch 188/300
144/144 - 0s - loss: 5.9708e-04
Epoch 189/300
144/144 - 0s - loss: 9.9484e-04
Epoch 190/300
144/144 - 0s - loss: 5.8326e-04
Epoch 191/300
144/144 - 0s - loss: 3.9766e-04
Epoch 192/300
144/144 - 0s - loss: 2.9902e-04
Epoch 193/300
144/144 - 0s - loss: 3.7086e-04
Epoch 194/300
144/144 - 0s - loss: 3.1093e-04
Epoch 195/300
144/144 - 0s - loss: 3.5426e-04
Epoch 196/300
144/144 - 0s - loss: 4.6887e-04
Epoch 197/300
144/144 - 0s - loss: 4.7468e-04
Epoch 198/300
144/144 - 0s - loss: 3.7654e-04
Epoch 199/300
144/144 - 0s - loss: 3.3881e-04
Epoch 200/300
144/144 - 0s - loss: 3.6496e-04
Epoch 201/300
144/144 - 0s - loss:

Epoch 50/300
144/144 - 0s - loss: 0.0031
Epoch 51/300
144/144 - 0s - loss: 0.0025
Epoch 52/300
144/144 - 0s - loss: 0.0021
Epoch 53/300
144/144 - 0s - loss: 0.0025
Epoch 54/300
144/144 - 0s - loss: 0.0027
Epoch 55/300
144/144 - 0s - loss: 0.0024
Epoch 56/300
144/144 - 0s - loss: 0.0021
Epoch 57/300
144/144 - 0s - loss: 0.0017
Epoch 58/300
144/144 - 0s - loss: 0.0018
Epoch 59/300
144/144 - 0s - loss: 0.0020
Epoch 60/300
144/144 - 0s - loss: 0.0019
Epoch 61/300
144/144 - 0s - loss: 0.0017
Epoch 62/300
144/144 - 0s - loss: 0.0019
Epoch 63/300
144/144 - 0s - loss: 0.0015
Epoch 64/300
144/144 - 0s - loss: 0.0016
Epoch 65/300
144/144 - 0s - loss: 0.0016
Epoch 66/300
144/144 - 0s - loss: 0.0016
Epoch 67/300
144/144 - 0s - loss: 0.0018
Epoch 68/300
144/144 - 0s - loss: 0.0014
Epoch 69/300
144/144 - 0s - loss: 0.0015
Epoch 70/300
144/144 - 0s - loss: 0.0016
Epoch 71/300
144/144 - 0s - loss: 0.0013
Epoch 72/300
144/144 - 0s - loss: 0.0013
Epoch 73/300
144/144 - 0s - loss: 0.0017
Epoch 74/300
144

Epoch 233/300
144/144 - 0s - loss: 3.0625e-04
Epoch 234/300
144/144 - 0s - loss: 1.7847e-04
Epoch 235/300
144/144 - 0s - loss: 1.9333e-04
Epoch 236/300
144/144 - 0s - loss: 1.3798e-04
Epoch 237/300
144/144 - 0s - loss: 1.2881e-04
Epoch 238/300
144/144 - 0s - loss: 1.8309e-04
Epoch 239/300
144/144 - 0s - loss: 2.3553e-04
Epoch 240/300
144/144 - 0s - loss: 1.7162e-04
Epoch 241/300
144/144 - 0s - loss: 2.0073e-04
Epoch 242/300
144/144 - 0s - loss: 1.7094e-04
Epoch 243/300
144/144 - 0s - loss: 2.1223e-04
Epoch 244/300
144/144 - 0s - loss: 7.9064e-04
Epoch 245/300
144/144 - 0s - loss: 4.0706e-04
Epoch 246/300
144/144 - 0s - loss: 1.3076e-04
Epoch 247/300
144/144 - 0s - loss: 1.1343e-04
Epoch 248/300
144/144 - 0s - loss: 1.0476e-04
Epoch 249/300
144/144 - 0s - loss: 9.1675e-05
Epoch 250/300
144/144 - 0s - loss: 1.5411e-04
Epoch 251/300
144/144 - 0s - loss: 1.0916e-04
Epoch 252/300
144/144 - 0s - loss: 9.4132e-05
Epoch 253/300
144/144 - 0s - loss: 9.9466e-05
Epoch 254/300
144/144 - 0s - loss:

Epoch 109/300
144/144 - 0s - loss: 0.0022
Epoch 110/300
144/144 - 0s - loss: 0.0022
Epoch 111/300
144/144 - 0s - loss: 0.0022
Epoch 112/300
144/144 - 0s - loss: 0.0022
Epoch 113/300
144/144 - 0s - loss: 0.0021
Epoch 114/300
144/144 - 0s - loss: 0.0023
Epoch 115/300
144/144 - 0s - loss: 0.0022
Epoch 116/300
144/144 - 0s - loss: 0.0021
Epoch 117/300
144/144 - 0s - loss: 0.0021
Epoch 118/300
144/144 - 0s - loss: 0.0022
Epoch 119/300
144/144 - 0s - loss: 0.0020
Epoch 120/300
144/144 - 0s - loss: 0.0020
Epoch 121/300
144/144 - 0s - loss: 0.0020
Epoch 122/300
144/144 - 0s - loss: 0.0016
Epoch 123/300
144/144 - 0s - loss: 0.0025
Epoch 124/300
144/144 - 0s - loss: 0.0021
Epoch 125/300
144/144 - 0s - loss: 0.0018
Epoch 126/300
144/144 - 0s - loss: 0.0018
Epoch 127/300
144/144 - 0s - loss: 0.0015
Epoch 128/300
144/144 - 0s - loss: 0.0016
Epoch 129/300
144/144 - 0s - loss: 0.0018
Epoch 130/300
144/144 - 0s - loss: 0.0016
Epoch 131/300
144/144 - 0s - loss: 0.0016
Epoch 132/300
144/144 - 0s - loss:

...

Epoch 34/300
144/144 - 0s - loss: 0.0146
Epoch 35/300
144/144 - 0s - loss: 0.0143
Epoch 36/300
144/144 - 0s - loss: 0.0144
Epoch 37/300
144/144 - 0s - loss: 0.0137
Epoch 38/300
144/144 - 0s - loss: 0.0134
Epoch 39/300
144/144 - 0s - loss: 0.0130
Epoch 40/300
144/144 - 0s - loss: 0.0129
Epoch 41/300
144/144 - 0s - loss: 0.0123
Epoch 42/300
144/144 - 0s - loss: 0.0124
Epoch 43/300
144/144 - 0s - loss: 0.0124
Epoch 44/300
144/144 - 0s - loss: 0.0116
Epoch 45/300
144/144 - 0s - loss: 0.0111
Epoch 46/300
144/144 - 0s - loss: 0.0110
Epoch 47/300
144/144 - 0s - loss: 0.0106
Epoch 48/300
144/144 - 0s - loss: 0.0107
Epoch 49/300
144/144 - 0s - loss: 0.0097
Epoch 50/300
144/144 - 0s - loss: 0.0095
Epoch 51/300
144/144 - 0s - loss: 0.0100
Epoch 52/300
144/144 - 0s - loss: 0.0086
Epoch 53/300
144/144 - 0s - loss: 0.0077
Epoch 54/300
144/144 - 0s - loss: 0.0075
Epoch 55/300
144/144 - 0s - loss: 0.0068
Epoch 56/300
144/144 - 0s - loss: 0.0072
Epoch 57/300
144/144 - 0s - loss: 0.0060
Epoch 58/300
144

Epoch 223/300
144/144 - 0s - loss: 2.0105e-04
Epoch 224/300
144/144 - 0s - loss: 1.9950e-04
Epoch 225/300
144/144 - 0s - loss: 2.4485e-04
Epoch 226/300
144/144 - 0s - loss: 2.1446e-04
Epoch 227/300
144/144 - 0s - loss: 1.8973e-04
Epoch 228/300
144/144 - 0s - loss: 1.8654e-04
Epoch 229/300
144/144 - 0s - loss: 1.9495e-04
Epoch 230/300
144/144 - 0s - loss: 2.2110e-04
Epoch 231/300
144/144 - 0s - loss: 2.5242e-04
Epoch 232/300
144/144 - 0s - loss: 2.2680e-04
Epoch 233/300
144/144 - 0s - loss: 2.6725e-04
Epoch 234/300
144/144 - 0s - loss: 3.1818e-04
Epoch 235/300
144/144 - 0s - loss: 3.0127e-04
Epoch 236/300
144/144 - 0s - loss: 2.4951e-04
Epoch 237/300
144/144 - 0s - loss: 2.8031e-04
Epoch 238/300
144/144 - 0s - loss: 2.3214e-04
Epoch 239/300
144/144 - 0s - loss: 3.6173e-04
Epoch 240/300
144/144 - 0s - loss: 5.4220e-04
Epoch 241/300
144/144 - 0s - loss: 4.4000e-04
Epoch 242/300
144/144 - 0s - loss: 3.0022e-04
Epoch 243/300
144/144 - 0s - loss: 2.8426e-04
Epoch 244/300
144/144 - 0s - loss:

Epoch 98/300
144/144 - 0s - loss: 0.0026
Epoch 99/300
144/144 - 0s - loss: 0.0029
Epoch 100/300
144/144 - 0s - loss: 0.0024
Epoch 101/300
144/144 - 0s - loss: 0.0027
Epoch 102/300
144/144 - 0s - loss: 0.0022
Epoch 103/300
144/144 - 0s - loss: 0.0022
Epoch 104/300
144/144 - 0s - loss: 0.0023
Epoch 105/300
144/144 - 0s - loss: 0.0022
Epoch 106/300
144/144 - 0s - loss: 0.0023
Epoch 107/300
144/144 - 0s - loss: 0.0023
Epoch 108/300
144/144 - 0s - loss: 0.0021
Epoch 109/300
144/144 - 0s - loss: 0.0019
Epoch 110/300
144/144 - 0s - loss: 0.0020
Epoch 111/300
144/144 - 0s - loss: 0.0021
Epoch 112/300
144/144 - 0s - loss: 0.0022
Epoch 113/300
144/144 - 0s - loss: 0.0023
Epoch 114/300
144/144 - 0s - loss: 0.0017
Epoch 115/300
144/144 - 0s - loss: 0.0019
Epoch 116/300
144/144 - 0s - loss: 0.0018
Epoch 117/300
144/144 - 0s - loss: 0.0018
Epoch 118/300
144/144 - 0s - loss: 0.0018
Epoch 119/300
144/144 - 0s - loss: 0.0018
Epoch 120/300
144/144 - 0s - loss: 0.0021
Epoch 121/300
144/144 - 0s - loss: 0

Epoch 282/300
144/144 - 0s - loss: 5.8024e-05
Epoch 283/300
144/144 - 0s - loss: 8.2130e-05
Epoch 284/300
144/144 - 0s - loss: 5.4488e-05
Epoch 285/300
144/144 - 0s - loss: 6.0656e-05
Epoch 286/300
144/144 - 0s - loss: 6.9350e-05
Epoch 287/300
144/144 - 0s - loss: 6.7280e-05
Epoch 288/300
144/144 - 0s - loss: 9.6838e-05
Epoch 289/300
144/144 - 0s - loss: 1.1034e-04
Epoch 290/300
144/144 - 0s - loss: 1.4440e-04
Epoch 291/300
144/144 - 0s - loss: 2.3677e-04
Epoch 292/300
144/144 - 0s - loss: 4.4275e-04
Epoch 293/300
144/144 - 0s - loss: 4.0824e-04
Epoch 294/300
144/144 - 0s - loss: 1.9780e-04
Epoch 295/300
144/144 - 0s - loss: 2.2734e-04
Epoch 296/300
144/144 - 0s - loss: 1.5803e-04
Epoch 297/300
144/144 - 0s - loss: 2.1641e-04
Epoch 298/300
144/144 - 0s - loss: 2.3229e-04
Epoch 299/300
144/144 - 0s - loss: 2.3459e-04
Epoch 300/300
144/144 - 0s - loss: 1.6782e-04
2022-01-12 06:55:03.491654
Model: "sequential_19"
_________________________________________________________________
Layer (typ

Epoch 160/300
144/144 - 0s - loss: 5.7144e-04
Epoch 161/300
144/144 - 0s - loss: 5.8415e-04
Epoch 162/300
144/144 - 0s - loss: 5.1604e-04
Epoch 163/300
144/144 - 0s - loss: 5.8113e-04
Epoch 164/300
144/144 - 0s - loss: 7.7131e-04
Epoch 165/300
144/144 - 0s - loss: 9.6397e-04
Epoch 166/300
144/144 - 0s - loss: 8.4480e-04
Epoch 167/300
144/144 - 0s - loss: 5.8065e-04
Epoch 168/300
144/144 - 0s - loss: 4.9066e-04
Epoch 169/300
144/144 - 0s - loss: 5.0047e-04
Epoch 170/300
144/144 - 0s - loss: 4.7526e-04
Epoch 171/300
144/144 - 0s - loss: 4.4162e-04
Epoch 172/300
144/144 - 0s - loss: 5.4493e-04
Epoch 173/300
144/144 - 0s - loss: 5.5449e-04
Epoch 174/300
144/144 - 0s - loss: 5.2622e-04
Epoch 175/300
144/144 - 0s - loss: 5.2465e-04
Epoch 176/300
144/144 - 0s - loss: 4.8769e-04
Epoch 177/300
144/144 - 0s - loss: 4.7349e-04
Epoch 178/300
144/144 - 0s - loss: 6.1227e-04
Epoch 179/300
144/144 - 0s - loss: 6.5395e-04
Epoch 180/300
144/144 - 0s - loss: 6.3757e-04
Epoch 181/300
144/144 - 0s - loss:

In [5]:
for station_id in stationid_set:
    train = main_df[main_df['station_id'] == station_id]
    train = train[train['date'] < '2014-06-01']
    train = train[train['date'] >= '2014-05-01']
    train = train.drop("station_id",axis=1)

    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2014-06-08 00:00:00']
    test = test[test['date'] >= '2014-05-31']
    test = test.drop("station_id",axis=1)

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    #0から1の範囲に値を正規化して配列に直しているのが下の2行
    train_df_scale = scaler.fit_transform(train.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(train[["bikes_available"]])
    
    length = len(train_df_scale)
    train = train_df_scale[0:length,:]

    def create_dataset(dataset):
        dataX = []
        dataY = np.array([])

        #trainまたはtestの行を計算している。
        max_len = len(dataset)
        for i in range(24,max_len):
            xset = []
            #trainまたはtestの列を繰り返しに入れている。
            for j in range(1, dataset.shape[1]):
                a = dataset[i-24:i, j]
                xset.append(a)
            #各行の正規化したバイク台数を取ってくる
            temp_array = np.array(dataset[i:i+1,0])
            #各行からバイク台数だけを取ってきたものをまとめている
            dataY = np.concatenate([dataY,temp_array])
            dataX.append(xset)
            #dataYの形を変えてるだけ
        dataY = dataY.reshape(-1,1)
        return np.array(dataX), dataY 

    trainX, trainY = create_dataset(train)
    #LSTMのモデルに入力用にデータの形を整形
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))

    model = Sequential()
    model.add(LSTM(50, input_shape=(trainX.shape[1],24)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    hist = model.fit(trainX, trainY, epochs=300, batch_size=5, verbose=2)
    
    #学習済みモデルで予測
    train_predict = model.predict(trainX)
    #スケールをもとに戻す
    train_predict = scaler_for_inverse.inverse_transform(train_predict)
    trainY = scaler_for_inverse.inverse_transform(trainY)
    #各ステーションのスコアの平均値を算出
    train_score_list = []
    trainscore = math.sqrt(mean_squared_error(trainY[:,0], train_predict[:,0]))
    train_score_list.append(trainscore)   
    
    array = []
    for i in train_predict:
        array.append(i[0])

    train = main_df[main_df['station_id'] == station_id]
    train = train[train['date'] < '2014-06-01']
    train = train[train['date'] >= '2014-05-02']
    
    train["predict"] =  array
    train_file_name = "LSTM_24_predict/lstm_train_stationid" + str(station_id) + "_201405.csv" 
    train.to_csv(train_file_name) 

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    test_df_scale = scaler.fit_transform(test.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(test[["bikes_available"]])
    
    length = len(test_df_scale)
    test = test_df_scale[0:length,:]
    testX, testY = create_dataset(test)
    #LSTMのモデルに入力用にデータの形を整形
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
    #学習済みモデルで予測
    test_predict = model.predict(testX)
    #スケールをもとに戻す
    test_predict = scaler_for_inverse.inverse_transform(test_predict)
    testY = scaler_for_inverse.inverse_transform(testY)
    #各ステーションのスコアの平均値を算出
    test_score_list = []
    testscore = math.sqrt(mean_squared_error(testY[:,0], test_predict[:,0]))
    test_score_list.append(testscore)

    array_pred = []
    for i in test_predict:
        array_pred.append(i[0])    
        
    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2014-06-08 00:00:00']
    test = test[test['date'] >= '2014-06-01']

    test['predict'] = array_pred
    test_file_name = "LSTM_24_predict/lstm_test_stationid" + str(station_id) + "_201405.csv" 
    test.to_csv(test_file_name) 

    dt_now = datetime.datetime.now()
    print(dt_now)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_20 (LSTM)               (None, 50)                15000     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 51        
Total params: 15,051
Trainable params: 15,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
144/144 - 1s - loss: 0.0347
Epoch 2/300
144/144 - 0s - loss: 0.0262
Epoch 3/300
144/144 - 0s - loss: 0.0247
Epoch 4/300
144/144 - 0s - loss: 0.0238
Epoch 5/300
144/144 - 0s - loss: 0.0229
Epoch 6/300
144/144 - 0s - loss: 0.0218
Epoch 7/300
144/144 - 0s - loss: 0.0219
Epoch 8/300
144/144 - 0s - loss: 0.0206
Epoch 9/300
144/144 - 0s - loss: 0.0205
Epoch 10/300
144/144 - 0s - loss: 0.0204
Epoch 11/300
144/144 - 0s - loss: 0.0200
Epoch 12/300
144/144 - 0s - loss: 0.0191
Epoch 13/300
144/144 - 0s - 

Epoch 179/300
144/144 - 0s - loss: 3.3841e-04
Epoch 180/300
144/144 - 0s - loss: 3.3057e-04
Epoch 181/300
144/144 - 0s - loss: 3.6938e-04
Epoch 182/300
144/144 - 0s - loss: 3.8461e-04
Epoch 183/300
144/144 - 0s - loss: 4.2427e-04
Epoch 184/300
144/144 - 0s - loss: 4.3052e-04
Epoch 185/300
144/144 - 0s - loss: 4.2653e-04
Epoch 186/300
144/144 - 0s - loss: 4.4602e-04
Epoch 187/300
144/144 - 0s - loss: 3.8751e-04
Epoch 188/300
144/144 - 0s - loss: 3.6397e-04
Epoch 189/300
144/144 - 0s - loss: 4.0757e-04
Epoch 190/300
144/144 - 0s - loss: 2.7386e-04
Epoch 191/300
144/144 - 0s - loss: 3.1811e-04
Epoch 192/300
144/144 - 0s - loss: 3.6595e-04
Epoch 193/300
144/144 - 0s - loss: 5.4845e-04
Epoch 194/300
144/144 - 0s - loss: 0.0015
Epoch 195/300
144/144 - 0s - loss: 6.4266e-04
Epoch 196/300
144/144 - 0s - loss: 3.0653e-04
Epoch 197/300
144/144 - 0s - loss: 2.5469e-04
Epoch 198/300
144/144 - 0s - loss: 2.5709e-04
Epoch 199/300
144/144 - 0s - loss: 2.5730e-04
Epoch 200/300
144/144 - 0s - loss: 2.6

Epoch 49/300
144/144 - 0s - loss: 0.0020
Epoch 50/300
144/144 - 0s - loss: 0.0016
Epoch 51/300
144/144 - 0s - loss: 0.0019
Epoch 52/300
144/144 - 0s - loss: 0.0022
Epoch 53/300
144/144 - 0s - loss: 0.0015
Epoch 54/300
144/144 - 0s - loss: 0.0016
Epoch 55/300
144/144 - 0s - loss: 0.0018
Epoch 56/300
144/144 - 0s - loss: 0.0013
Epoch 57/300
144/144 - 0s - loss: 0.0014
Epoch 58/300
144/144 - 0s - loss: 0.0017
Epoch 59/300
144/144 - 0s - loss: 0.0015
Epoch 60/300
144/144 - 0s - loss: 0.0015
Epoch 61/300
144/144 - 0s - loss: 0.0016
Epoch 62/300
144/144 - 0s - loss: 0.0012
Epoch 63/300
144/144 - 0s - loss: 0.0011
Epoch 64/300
144/144 - 0s - loss: 0.0015
Epoch 65/300
144/144 - 0s - loss: 0.0014
Epoch 66/300
144/144 - 0s - loss: 0.0013
Epoch 67/300
144/144 - 0s - loss: 0.0012
Epoch 68/300
144/144 - 0s - loss: 0.0014
Epoch 69/300
144/144 - 0s - loss: 0.0010
Epoch 70/300
144/144 - 0s - loss: 9.4378e-04
Epoch 71/300
144/144 - 0s - loss: 0.0010
Epoch 72/300
144/144 - 0s - loss: 0.0012
Epoch 73/300

Epoch 232/300
144/144 - 0s - loss: 2.0753e-04
Epoch 233/300
144/144 - 0s - loss: 1.8073e-04
Epoch 234/300
144/144 - 0s - loss: 3.0180e-04
Epoch 235/300
144/144 - 0s - loss: 2.1378e-04
Epoch 236/300
144/144 - 0s - loss: 1.7087e-04
Epoch 237/300
144/144 - 0s - loss: 2.2200e-04
Epoch 238/300
144/144 - 0s - loss: 2.2902e-04
Epoch 239/300
144/144 - 0s - loss: 2.8016e-04
Epoch 240/300
144/144 - 0s - loss: 1.7998e-04
Epoch 241/300
144/144 - 0s - loss: 1.6470e-04
Epoch 242/300
144/144 - 0s - loss: 1.6447e-04
Epoch 243/300
144/144 - 0s - loss: 2.0295e-04
Epoch 244/300
144/144 - 0s - loss: 2.1265e-04
Epoch 245/300
144/144 - 0s - loss: 2.0799e-04
Epoch 246/300
144/144 - 0s - loss: 2.2790e-04
Epoch 247/300
144/144 - 0s - loss: 1.7973e-04
Epoch 248/300
144/144 - 0s - loss: 1.6419e-04
Epoch 249/300
144/144 - 0s - loss: 1.6006e-04
Epoch 250/300
144/144 - 0s - loss: 1.6010e-04
Epoch 251/300
144/144 - 0s - loss: 2.5152e-04
Epoch 252/300
144/144 - 0s - loss: 1.9440e-04
Epoch 253/300
144/144 - 0s - loss:

Epoch 108/300
144/144 - 0s - loss: 0.0017
Epoch 109/300
144/144 - 0s - loss: 0.0016
Epoch 110/300
144/144 - 0s - loss: 0.0017
Epoch 111/300
144/144 - 0s - loss: 0.0020
Epoch 112/300
144/144 - 0s - loss: 0.0017
Epoch 113/300
144/144 - 0s - loss: 0.0017
Epoch 114/300
144/144 - 0s - loss: 0.0016
Epoch 115/300
144/144 - 0s - loss: 0.0018
Epoch 116/300
144/144 - 0s - loss: 0.0016
Epoch 117/300
144/144 - 0s - loss: 0.0015
Epoch 118/300
144/144 - 0s - loss: 0.0014
Epoch 119/300
144/144 - 0s - loss: 0.0014
Epoch 120/300
144/144 - 0s - loss: 0.0012
Epoch 121/300
144/144 - 0s - loss: 0.0013
Epoch 122/300
144/144 - 0s - loss: 0.0014
Epoch 123/300
144/144 - 0s - loss: 0.0015
Epoch 124/300
144/144 - 0s - loss: 0.0014
Epoch 125/300
144/144 - 0s - loss: 0.0014
Epoch 126/300
144/144 - 0s - loss: 0.0016
Epoch 127/300
144/144 - 0s - loss: 0.0013
Epoch 128/300
144/144 - 0s - loss: 0.0010
Epoch 129/300
144/144 - 0s - loss: 9.9096e-04
Epoch 130/300
144/144 - 0s - loss: 9.7558e-04
Epoch 131/300
144/144 - 0s

...

Epoch 30/300
144/144 - 0s - loss: 0.0358
Epoch 31/300
144/144 - 0s - loss: 0.0366
Epoch 32/300
144/144 - 0s - loss: 0.0344
Epoch 33/300
144/144 - 0s - loss: 0.0345
Epoch 34/300
144/144 - 0s - loss: 0.0313
Epoch 35/300
144/144 - 0s - loss: 0.0289
Epoch 36/300
144/144 - 0s - loss: 0.0283
Epoch 37/300
144/144 - 0s - loss: 0.0248
Epoch 38/300
144/144 - 0s - loss: 0.0258
Epoch 39/300
144/144 - 0s - loss: 0.0216
Epoch 40/300
144/144 - 0s - loss: 0.0189
Epoch 41/300
144/144 - 0s - loss: 0.0177
Epoch 42/300
144/144 - 0s - loss: 0.0153
Epoch 43/300
144/144 - 0s - loss: 0.0144
Epoch 44/300
144/144 - 0s - loss: 0.0137
Epoch 45/300
144/144 - 0s - loss: 0.0126
Epoch 46/300
144/144 - 0s - loss: 0.0115
Epoch 47/300
144/144 - 0s - loss: 0.0119
Epoch 48/300
144/144 - 0s - loss: 0.0115
Epoch 49/300
144/144 - 0s - loss: 0.0102
Epoch 50/300
144/144 - 0s - loss: 0.0096
Epoch 51/300
144/144 - 0s - loss: 0.0089
Epoch 52/300
144/144 - 0s - loss: 0.0086
Epoch 53/300
144/144 - 0s - loss: 0.0084
Epoch 54/300
144

Epoch 221/300
144/144 - 0s - loss: 4.6218e-04
Epoch 222/300
144/144 - 0s - loss: 4.2865e-04
Epoch 223/300
144/144 - 0s - loss: 3.5911e-04
Epoch 224/300
144/144 - 0s - loss: 7.1625e-04
Epoch 225/300
144/144 - 0s - loss: 4.8962e-04
Epoch 226/300
144/144 - 0s - loss: 4.4325e-04
Epoch 227/300
144/144 - 0s - loss: 2.9901e-04
Epoch 228/300
144/144 - 0s - loss: 3.5727e-04
Epoch 229/300
144/144 - 0s - loss: 4.1865e-04
Epoch 230/300
144/144 - 0s - loss: 6.0946e-04
Epoch 231/300
144/144 - 0s - loss: 0.0036
Epoch 232/300
144/144 - 0s - loss: 0.0027
Epoch 233/300
144/144 - 0s - loss: 4.4810e-04
Epoch 234/300
144/144 - 0s - loss: 2.7709e-04
Epoch 235/300
144/144 - 0s - loss: 2.4680e-04
Epoch 236/300
144/144 - 0s - loss: 2.2233e-04
Epoch 237/300
144/144 - 0s - loss: 2.2947e-04
Epoch 238/300
144/144 - 0s - loss: 1.9564e-04
Epoch 239/300
144/144 - 0s - loss: 1.6998e-04
Epoch 240/300
144/144 - 0s - loss: 2.1132e-04
Epoch 241/300
144/144 - 0s - loss: 1.8978e-04
Epoch 242/300
144/144 - 0s - loss: 1.9074e

Epoch 96/300
144/144 - 0s - loss: 0.0021
Epoch 97/300
144/144 - 0s - loss: 0.0023
Epoch 98/300
144/144 - 0s - loss: 0.0025
Epoch 99/300
144/144 - 0s - loss: 0.0025
Epoch 100/300
144/144 - 0s - loss: 0.0023
Epoch 101/300
144/144 - 0s - loss: 0.0024
Epoch 102/300
144/144 - 0s - loss: 0.0022
Epoch 103/300
144/144 - 0s - loss: 0.0021
Epoch 104/300
144/144 - 0s - loss: 0.0039
Epoch 105/300
144/144 - 0s - loss: 0.0018
Epoch 106/300
144/144 - 0s - loss: 0.0019
Epoch 107/300
144/144 - 0s - loss: 0.0019
Epoch 108/300
144/144 - 0s - loss: 0.0019
Epoch 109/300
144/144 - 0s - loss: 0.0021
Epoch 110/300
144/144 - 0s - loss: 0.0019
Epoch 111/300
144/144 - 0s - loss: 0.0021
Epoch 112/300
144/144 - 0s - loss: 0.0018
Epoch 113/300
144/144 - 0s - loss: 0.0023
Epoch 114/300
144/144 - 0s - loss: 0.0022
Epoch 115/300
144/144 - 0s - loss: 0.0018
Epoch 116/300
144/144 - 0s - loss: 0.0018
Epoch 117/300
144/144 - 0s - loss: 0.0017
Epoch 118/300
144/144 - 0s - loss: 0.0015
Epoch 119/300
144/144 - 0s - loss: 0.0

Epoch 279/300
144/144 - 0s - loss: 3.0975e-04
Epoch 280/300
144/144 - 0s - loss: 2.1536e-04
Epoch 281/300
144/144 - 0s - loss: 2.0364e-04
Epoch 282/300
144/144 - 0s - loss: 2.3742e-04
Epoch 283/300
144/144 - 0s - loss: 3.5154e-04
Epoch 284/300
144/144 - 0s - loss: 3.0596e-04
Epoch 285/300
144/144 - 0s - loss: 2.6847e-04
Epoch 286/300
144/144 - 0s - loss: 2.4753e-04
Epoch 287/300
144/144 - 0s - loss: 2.4586e-04
Epoch 288/300
144/144 - 0s - loss: 2.3519e-04
Epoch 289/300
144/144 - 0s - loss: 3.2089e-04
Epoch 290/300
144/144 - 0s - loss: 3.1367e-04
Epoch 291/300
144/144 - 0s - loss: 6.2241e-04
Epoch 292/300
144/144 - 0s - loss: 4.3215e-04
Epoch 293/300
144/144 - 0s - loss: 2.5789e-04
Epoch 294/300
144/144 - 0s - loss: 2.0220e-04
Epoch 295/300
144/144 - 0s - loss: 2.5071e-04
Epoch 296/300
144/144 - 0s - loss: 2.4359e-04
Epoch 297/300
144/144 - 0s - loss: 1.9948e-04
Epoch 298/300
144/144 - 0s - loss: 3.4707e-04
Epoch 299/300
144/144 - 0s - loss: 3.1267e-04
Epoch 300/300
144/144 - 0s - loss:

Epoch 160/300
144/144 - 0s - loss: 0.0011
Epoch 161/300
144/144 - 0s - loss: 0.0010
Epoch 162/300
144/144 - 0s - loss: 0.0014
Epoch 163/300
144/144 - 0s - loss: 0.0011
Epoch 164/300
144/144 - 0s - loss: 0.0010
Epoch 165/300
144/144 - 0s - loss: 0.0017
Epoch 166/300
144/144 - 0s - loss: 0.0016
Epoch 167/300
144/144 - 0s - loss: 0.0012
Epoch 168/300
144/144 - 0s - loss: 0.0010
Epoch 169/300
144/144 - 0s - loss: 9.2709e-04
Epoch 170/300
144/144 - 0s - loss: 9.3312e-04
Epoch 171/300
144/144 - 0s - loss: 9.4725e-04
Epoch 172/300
144/144 - 0s - loss: 8.2566e-04
Epoch 173/300
144/144 - 0s - loss: 0.0011
Epoch 174/300
144/144 - 0s - loss: 0.0010
Epoch 175/300
144/144 - 0s - loss: 9.1554e-04
Epoch 176/300
144/144 - 0s - loss: 0.0013
Epoch 177/300
144/144 - 0s - loss: 0.0011
Epoch 178/300
144/144 - 0s - loss: 8.1128e-04
Epoch 179/300
144/144 - 0s - loss: 7.7004e-04
Epoch 180/300
144/144 - 0s - loss: 9.3143e-04
Epoch 181/300
144/144 - 0s - loss: 8.6471e-04
Epoch 182/300
144/144 - 0s - loss: 7.777

In [3]:
stationid_set =[0]

In [6]:
start = time.time()
for station_id in stationid_set:
    train = main_df[main_df['station_id'] == station_id]
    train = train[train['date'] < '2013-10-01']
    train = train[train['date'] >= '2013-09-01']
    train = train.drop("station_id",axis=1)

    test = main_df[main_df['station_id'] == station_id]
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-09-30']
    test = test.drop("station_id",axis=1)

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    #0から1の範囲に値を正規化して配列に直しているのが下の2行
    train_df_scale = scaler.fit_transform(train.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(train[["bikes_available"]])
    
    length = len(train_df_scale)
    train = train_df_scale[0:length,:]

    def create_dataset(dataset):
        dataX = []
        dataY = np.array([])

        #trainまたはtestの行を計算している。
        max_len = len(dataset)
        for i in range(24,max_len):
            xset = []
            #trainまたはtestの列を繰り返しに入れている。
            for j in range(1, dataset.shape[1]):
                a = dataset[i-24:i, j]
                xset.append(a)
            #各行の正規化したバイク台数を取ってくる
            temp_array = np.array(dataset[i:i+1,0])
            #各行からバイク台数だけを取ってきたものをまとめている
            dataY = np.concatenate([dataY,temp_array])
            dataX.append(xset)
            #dataYの形を変えてるだけ
        dataY = dataY.reshape(-1,1)
        return np.array(dataX), dataY 

    trainX, trainY = create_dataset(train)
    #LSTMのモデルに入力用にデータの形を整形
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))

    model = Sequential()
    model.add(LSTM(50, input_shape=(trainX.shape[1],24)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.summary()
    
    hist = model.fit(trainX, trainY, epochs=300, batch_size=5, verbose=2)
    
    #学習済みモデルで予測
    train_predict = model.predict(trainX)
    #スケールをもとに戻す
    train_predict = scaler_for_inverse.inverse_transform(train_predict)
    trainY = scaler_for_inverse.inverse_transform(trainY)
    #各ステーションのスコアの平均値を算出
    train_score_list = []
    trainscore = math.sqrt(mean_squared_error(trainY[:,0], train_predict[:,0]))
    train_score_list.append(trainscore)   
    
    array = []
    for i in train_predict:
        array.append(i[0])

    train = main_df[main_df['station_id'] == 0]
    train = train[train['date'] < '2013-10-01']
    train = train[train['date'] >= '2013-09-02']
    
    train["predict"] =  array
    train_file_name = "LSTM_24_predict/lstm_train_stationid" + str(station_id) + "_201309.csv" 
    train.to_csv(train_file_name) 

    #今回LSTMモデルを使用するため、データを標準化
    #特徴量を標準化するための変数
    scaler = MinMaxScaler(feature_range=(0, 1))
    #標準化された出力をもとにスケールに変換(inverse)するために必要な変数
    scaler_for_inverse = MinMaxScaler(feature_range=(0, 1))
    test_df_scale = scaler.fit_transform(test.iloc[:,2:])
    bikes_available_scale = scaler_for_inverse.fit_transform(test[["bikes_available"]])
    
    length = len(test_df_scale)
    test = test_df_scale[0:length,:]
    testX, testY = create_dataset(test)
    #LSTMのモデルに入力用にデータの形を整形
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))
    #学習済みモデルで予測
    test_predict = model.predict(testX)
    #スケールをもとに戻す
    test_predict = scaler_for_inverse.inverse_transform(test_predict)
    testY = scaler_for_inverse.inverse_transform(testY)
    #各ステーションのスコアの平均値を算出
    test_score_list = []
    testscore = math.sqrt(mean_squared_error(testY[:,0], test_predict[:,0]))
    test_score_list.append(testscore)

    array_pred = []
    for i in test_predict:
        array_pred.append(i[0])    
        
    test = main_df[main_df['station_id'] == 0]
    test = test[test['date'] <= '2013-10-08 00:00:00']
    test = test[test['date'] >= '2013-10-01']

    test['predict'] = array_pred
    test_file_name = "LSTM_24_predict/lstm_test_stationid" + str(station_id) + "_201309.csv" 
    test.to_csv(test_file_name)
    
elapsed_time = time.time() - start
print(elapsed_time)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                15000     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 15,051
Trainable params: 15,051
Non-trainable params: 0
_________________________________________________________________
Epoch 1/300
140/140 - 1s - loss: 0.0382
Epoch 2/300
140/140 - 0s - loss: 0.0263
Epoch 3/300
140/140 - 0s - loss: 0.0265
Epoch 4/300
140/140 - 0s - loss: 0.0242
Epoch 5/300
140/140 - 0s - loss: 0.0233
Epoch 6/300
140/140 - 0s - loss: 0.0221
Epoch 7/300
140/140 - 0s - loss: 0.0208
Epoch 8/300
140/140 - 0s - loss: 0.0196
Epoch 9/300
140/140 - 0s - loss: 0.0183
Epoch 10/300
140/140 - 0s - loss: 0.0178
Epoch 11/300
140/140 - 0s - loss: 0.0172
Epoch 12/300
140/140 - 0s - loss: 0.0172
Epoch 13/300
140/140 - 0s - los

140/140 - 0s - loss: 4.2239e-04
Epoch 178/300
140/140 - 0s - loss: 4.9859e-04
Epoch 179/300
140/140 - 0s - loss: 3.4177e-04
Epoch 180/300
140/140 - 0s - loss: 3.0654e-04
Epoch 181/300
140/140 - 0s - loss: 3.0381e-04
Epoch 182/300
140/140 - 0s - loss: 3.0006e-04
Epoch 183/300
140/140 - 0s - loss: 4.5689e-04
Epoch 184/300
140/140 - 0s - loss: 5.2666e-04
Epoch 185/300
140/140 - 0s - loss: 2.7994e-04
Epoch 186/300
140/140 - 0s - loss: 2.7565e-04
Epoch 187/300
140/140 - 0s - loss: 3.2812e-04
Epoch 188/300
140/140 - 0s - loss: 3.0026e-04
Epoch 189/300
140/140 - 0s - loss: 9.6073e-04
Epoch 190/300
140/140 - 0s - loss: 5.4019e-04
Epoch 191/300
140/140 - 0s - loss: 3.6754e-04
Epoch 192/300
140/140 - 0s - loss: 3.7002e-04
Epoch 193/300
140/140 - 0s - loss: 3.2003e-04
Epoch 194/300
140/140 - 0s - loss: 2.3570e-04
Epoch 195/300
140/140 - 0s - loss: 2.3039e-04
Epoch 196/300
140/140 - 0s - loss: 3.3135e-04
Epoch 197/300
140/140 - 0s - loss: 2.4838e-04
Epoch 198/300
140/140 - 0s - loss: 2.3877e-04
Ep